In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
import json
import tensorflowjs as tfjs

## Import Text

In [ ]:
with open("./trainingData/sherlockS.txt", 'r', encoding='utf-8') as myfile:
    mytext = myfile.read()

## Tokenize Text

In [ ]:
mytokenizer = Tokenizer()
mytokenizer.fit_on_texts([mytext])
total_words = len(mytokenizer.word_index) + 1

In [ ]:
my_input_sequences = []
for line in mytext.split('\n'):
    token_list = mytokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        my_n_gram_sequence = token_list[:i+1]
        my_input_sequences.append(my_n_gram_sequence)

In [ ]:
max_sequence_len = max([len(seq) for seq in my_input_sequences])
input_sequences = np.array(pad_sequences(my_input_sequences, maxlen=max_sequence_len, padding='pre'))

In [19]:
X = input_sequences[:, :-1]
y = input_sequences[:, 1:]

In [20]:
y = np.array(tf.keras.utils.to_categorical(y, num_classes=total_words))

## Create Model Architecture

In [36]:
model = Sequential()
model.add(Dense(150, input_shape=(max_sequence_len-1,), activation='relu'))
model.add(Dense(150, activation='relu'))
model.add(Dense(total_words, activation='softmax'))
print(model.summary())

TypeError: ('Keyword argument not understood:', 'input_length')

## Train Model

In [35]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X, y, epochs=100, verbose=1)

Epoch 1/100
71/71 [==============================] - 1s 7ms/step - loss: 5.4822 - accuracy: 0.5600
Epoch 2/100
71/71 [==============================] - 0s 7ms/step - loss: 2.5020 - accuracy: 0.5915
Epoch 3/100
71/71 [==============================] - 0s 7ms/step - loss: 2.2336 - accuracy: 0.6135
Epoch 4/100
71/71 [==============================] - 0s 7ms/step - loss: 1.9944 - accuracy: 0.6430
Epoch 5/100
71/71 [==============================] - 0s 6ms/step - loss: 1.7335 - accuracy: 0.6889
Epoch 6/100
71/71 [==============================] - 0s 6ms/step - loss: 1.5069 - accuracy: 0.7276
Epoch 7/100
71/71 [==============================] - 0s 6ms/step - loss: 1.3628 - accuracy: 0.7385
Epoch 8/100
71/71 [==============================] - 0s 6ms/step - loss: 1.2860 - accuracy: 0.7418
Epoch 9/100
71/71 [==============================] - 0s 7ms/step - loss: 1.2456 - accuracy: 0.7429
Epoch 10/100
71/71 [==============================] - 0s 6ms/step - loss: 1.2184 - accuracy: 0.7437
Epoch 11/

## Save Model for JS

In [23]:

output_dir = "./tfjs_models/FFNN" 
tfjs.converters.save_keras_model(model, output_dir)

C:\Users\lusti\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


## Save word index

In [24]:
wordIndex = {}
for word in mytokenizer.word_index:
    wordIndex[word] = mytokenizer.word_index[word]

with open('./tfjs_models/FFNN/wordIndex.json', 'w', encoding='utf-8') as f:
    f.write(json.dumps(wordIndex, ensure_ascii=False))

## Save Max Sequence Length

In [25]:
with open('./tfjs_models/FFNN/max_sequence_len.txt', 'w') as f:
    f.write(str(max_sequence_len))